In [1]:
import numpy as np
from scipy.cluster.vq import whiten, vq, kmeans, kmeans2

#### dataset train vectors with d = 6

In [15]:
data = np.array([[1, 2, 2, 1, 1, 2],
               [7, 8, 9, 10, 11, 12],
               [1, 4, 1, 2, 1, 3],
               [9, 11, 7, 6, 2, 1], 
               [7, 2, 1, 2, 3, 1], 
               [4, 6, 2, 5, 1, 4], 
               [2, 5, 7, 11, 1, 8],
               [4, 1, 1, 2, 6, 3],
               [3, 10, 2, 6, 1, 1],
               [1, 2, 1, 2, 3, 2],
               [8, 12, 1, 6, 10, 2],
               [12, 11, 8, 10, 11, 12] ])
data.shape

(12, 6)

#### query vector

In [16]:
query_x = np.array([[2, 1, 2, 1, 3, 1]])

#### correct neigbours for comparison

In [17]:
from sklearn.neighbors import NearestNeighbors
k = 5
true_classifier = NearestNeighbors(n_neighbors=k, metric='l2', algorithm='brute').fit(data)
corrects = true_classifier.kneighbors(query_x)

In [18]:
corrects[1] 

array([[9, 0, 7, 2, 4]])

#### 1. Divide each vector into subvector, whiten it and collect all together for clustering
 subvectors - 3 --> each subvector is 2-d 

In [19]:
n_subvectors = 3

In [20]:
all_subvectors = []
for x in data:
    whiten_x = whiten(x)
    x_subvectors = np.split(whiten_x, n_subvectors)
    all_subvectors += x_subvectors

In [21]:
all_subvectors[:5]

[array([2., 4.]),
 array([4., 2.]),
 array([2., 4.]),
 array([4.09878031, 4.68432035]),
 array([5.26986039, 5.85540044])]

#### 2.Obtain centroids using kmeans
number of centroids - 6

In [22]:
n_centroids = 6

In [23]:
centroids = kmeans(all_subvectors, n_centroids)

In [24]:
centroids 

(array([[0.63297879, 1.07989119],
        [0.95307395, 2.82356166],
        [2.92072354, 1.23637004],
        [8.36747906, 8.36747906],
        [2.67274653, 3.55170563],
        [5.84388529, 6.71931657]]),
 0.7084054240877972)

 the second value is mean val of dists from closest point in each cluster

In [25]:
centroids = centroids[0]

#### 3. Assign to subvectors of each vector indices of centrods

In [26]:
quantized_xs = []
for x in data:
    whiten_x = whiten(x)
    x_subvectors = np.array(np.split(whiten_x, 3))
    quantized_x = vq(x_subvectors, centroids)[0]
    quantized_xs.append(quantized_x)

In [27]:
quantized_xs

[array([4, 2, 4], dtype=int32),
 array([4, 5, 5], dtype=int32),
 array([1, 0, 1], dtype=int32),
 array([4, 2, 0], dtype=int32),
 array([2, 0, 0], dtype=int32),
 array([4, 1, 1], dtype=int32),
 array([0, 4, 1], dtype=int32),
 array([2, 0, 2], dtype=int32),
 array([1, 0, 0], dtype=int32),
 array([1, 1, 4], dtype=int32),
 array([4, 0, 2], dtype=int32),
 array([3, 5, 3], dtype=int32)]

#### 4. Build lookup tables of distances between all centrods
for later fast dist calculation with query vector

In [28]:
lookup_table = np.zeros(shape=(n_centroids,n_centroids))

In [29]:
from scipy.spatial import distance
for i, centroid1 in enumerate(centroids):
    for j, centroid2 in enumerate(centroids):
        lookup_table[i,j] = distance.euclidean(centroid1, centroid2)
        

In [30]:
lookup_table

array([[ 0.        ,  1.77280783,  2.29308998, 10.62692013,  3.20476505,
         7.67832437],
       [ 1.77280783,  0.        ,  2.52800753,  9.25788439,  1.86747622,
         6.25275474],
       [ 2.29308998,  2.52800753,  0.        ,  8.973286  ,  2.32857713,
         6.2134996 ],
       [10.62692013,  9.25788439,  8.973286  ,  0.        ,  7.45799252,
         3.01412759],
       [ 3.20476505,  1.86747622,  2.32857713,  7.45799252,  0.        ,
         4.48217359],
       [ 7.67832437,  6.25275474,  6.2134996 ,  3.01412759,  4.48217359,
         0.        ]])

#### 5. Proccess search of k neigbours for query vector

In [31]:
query_x = np.array([2, 1, 2, 1, 3, 1])

##### 5.1 Quantize query similiar to train vectors

In [32]:
whiten_query = whiten(query_x)
query_subvectors = np.split(whiten_query, n_subvectors)
quantized_query = vq(query_subvectors, centroids)[0]

In [33]:
quantized_query

array([2, 2, 2], dtype=int32)

#### Calculate distance beetween quantized query and dataset vectors using lookup table
we know all distances between centroids, so just get 2 indices and see in the table. 
<br>
it's Product Quantization --> all distances of subvectors are multiplied for getting vectors' distance

In [34]:
eps = 0.00001
dists=[]
for ind, quantized_x in enumerate(quantized_xs):
    dist = 1
    for i,j in zip(quantized_query,quantized_x):
        dist*= (lookup_table[i,j]+eps)
    dists.append((dist, ind))

In [35]:
nearest_inds = [ind for _, ind in sorted(dists, key=lambda x: x[0])[:k]]

#### result of PQ:

In [36]:
nearest_inds

[7, 4, 10, 3, 0]

#### result of true scikit learn knn Classifier: 

In [37]:
corrects[1] 

array([[9, 0, 7, 2, 4]])

In [39]:
nearests = [data[i] for i in nearest_inds]

In [40]:
print(query_x)
print('-'*50)
nearests

[2 1 2 1 3 1]
--------------------------------------------------


[array([4, 1, 1, 2, 6, 3]),
 array([7, 2, 1, 2, 3, 1]),
 array([ 8, 12,  1,  6, 10,  2]),
 array([ 9, 11,  7,  6,  2,  1]),
 array([1, 2, 2, 1, 1, 2])]

<hr>

In [1]:
from product_quantization import PQ
import numpy as np
from test import test_index

In [2]:
data = np.array([[1, 2, 2, 1, 1, 2],
               [7, 8, 9, 10, 11, 12],
               [1, 4, 1, 2, 1, 3],
               [9, 11, 7, 6, 2, 1], 
               [7, 2, 1, 2, 3, 1], 
               [4, 6, 2, 5, 1, 4], 
               [2, 5, 7, 11, 1, 8],
               [4, 1, 1, 2, 6, 3],
               [3, 10, 2, 6, 1, 1],
               [1, 2, 1, 2, 3, 2],
               [8, 12, 1, 6, 10, 2],
               [12, 11, 8, 10, 11, 12] ])
data

array([[ 1,  2,  2,  1,  1,  2],
       [ 7,  8,  9, 10, 11, 12],
       [ 1,  4,  1,  2,  1,  3],
       [ 9, 11,  7,  6,  2,  1],
       [ 7,  2,  1,  2,  3,  1],
       [ 4,  6,  2,  5,  1,  4],
       [ 2,  5,  7, 11,  1,  8],
       [ 4,  1,  1,  2,  6,  3],
       [ 3, 10,  2,  6,  1,  1],
       [ 1,  2,  1,  2,  3,  2],
       [ 8, 12,  1,  6, 10,  2],
       [12, 11,  8, 10, 11, 12]])

In [3]:
query = np.array([[2, 1, 2, 1, 3, 1], [11, 10, 9, 8, 4, 1]])


In [4]:
params = {'n_subvectors': 3, 'n_bits':3 }
test_index('PQ',  data=data, query=query, k=3, params=params)

{'method': 'PQ',
 'construction-time': 1.16,
 'construction-memory': 0.61,
 'search-time': 0.043,
 'search-memory': 286.72,
 'recall': 0.83}